In [2]:
from IPython.display import display, HTML


def visualize_state(state):
    """Visualizes the given state of the Taquin using HTML."""

    html = "<table>"

    for row in state:

        html += "<tr>"

        for tile in row:

            if tile == 0:

                html += "<td style='background-color: lightgray; width: 30px; height: 30px; text-align: center; font-size: 20px;'> </td>"  # Blank tile

            else:

                html += f"<td style='background-color: lightblue; width: 30px; height: 30px; text-align: center; font-size: 20px; color: black;'>{tile}</td>"

        html += "</tr>"

    html += "</table>"

    display(HTML(html))


class Taquin:
    """

    A class representing the Taquin problem.
    """

    def __init__(self, initial_state, goal_state, size):

        self.initial_state = initial_state

        self.goal_state = goal_state

        self.size = size

    def actions(self, state):
        """Returns the possible actions (moves) from the given state."""

        # Find the position of the blank tile (0)

        row, col = next(
            (r, c)
            for r, row in enumerate(state)
            for c, val in enumerate(row)
            if val == 0
        )

        # Define possible moves (up, down, left, right)

        possible_actions = []

        if row > 0:

            possible_actions.append("up")

        if row < self.size - 1:

            possible_actions.append("down")

        if col > 0:

            possible_actions.append("left")

        if col < self.size - 1:

            possible_actions.append("right")

        return possible_actions

    def result(self, state, action):
        """Returns the state that results from applying the given action."""

        # Create a copy of the state to avoid modifying the original

        new_state = [list(row) for row in state]

        # Find the position of the blank tile (0)

        row, col = next(
            (r, c)
            for r, row in enumerate(state)
            for c, val in enumerate(row)
            if val == 0
        )

        # Apply the action to move the blank tile

        if action == "up":

            new_state[row][col], new_state[row - 1][col] = (
                new_state[row - 1][col],
                new_state[row][col],
            )

        elif action == "down":

            new_state[row][col], new_state[row + 1][col] = (
                new_state[row + 1][col],
                new_state[row][col],
            )

        elif action == "left":

            new_state[row][col], new_state[row][col - 1] = (
                new_state[row][col - 1],
                new_state[row][col],
            )

        elif action == "right":

            new_state[row][col], new_state[row][col + 1] = (
                new_state[row][col + 1],
                new_state[row][col],
            )

        return new_state

    def is_goal(self, state):

        return state == self.goal_state  # Directly compare with goal_state

    def cost(self, state, action):

        return 1  # Default cost is 1


class Node:
    """

    A node in a search tree.

    __init__: Initializes a node with its state, parent, action, path cost, and depth.

    __repr__: Provides a string representation of the node.

    __lt__: Defines a comparison operator for nodes based on their states.

    expand: Generates child nodes by applying all possible actions.

    child_node: Creates a single child node for a given action.

    solution: Returns the sequence of actions that led to this node.

    path: Returns the path from the root to this node as a list of nodes.
    """

    def __init__(self, state, parent=None, action=None, path_cost=0):

        self.state = state

        self.parent = parent

        self.action = action

        self.path_cost = path_cost

        self.depth = 0 if parent is None else parent.depth + 1

    # def __repr__(self):

    #     return "<Node {}>".format(self.state)

    # def __lt__(self, other):

    #     return self.state < other.state

    def __hash__(self):
        """Convert state to a tuple so it can be hashed"""
        return hash(
            tuple(map(tuple, self.state))
        )  # Convert list of lists to tuple of tuples

    def __eq__(self, other):
        """Check if two nodes have the same state"""
        return isinstance(other, Node) and self.state == other.state

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""

        return [
            self.child_node(problem, action) for action in problem.actions(self.state)
        ]

    def child_node(self, problem, action):
        """Create a child node by applying the given action."""

        next_state = problem.result(self.state, action)

        next_node = Node(
            next_state,
            parent=self,
            action=action,
            path_cost=self.path_cost + problem.cost(self.state, action),
        )

        return next_node

    def solution(self):
        """Return the sequence of actions to go from the root to this node."""

        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""

        node, path_back = self, []

        while node:

            path_back.append(node)

            node = node.parent

        return list(reversed(path_back))

Question 1. Importez le fichier Excel taquin_1000_states sur Colab

In [15]:
import pandas as pd
import numpy as np

taquins = pd.read_excel("taquin_1000_states.xlsx")

print(taquins.head())

visualize_state(grids[0])

intial_state_test = [
    [2, 1, 4],
    [5, 0, 6],
    [8, 3, 7],
]
goal_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0],
]

   col0  col1  col2  col3  col4  col5  col6  col7  col8  h1  h2  h3   h4  \
0     7     5     3     2     6     1     8     4     0   7  12  12  7.2   
1     1     4     3     6     7     8     5     0     2   6  13  13  6.2   
2     1     2     0     8     3     7     5     6     4   6  14  14  6.2   
3     1     6     2     7     3     0     8     5     4   7  11  11  7.4   
4     1     4     5     6     2     8     3     7     0   7  14  14  7.2   

   target  
0      22  
1      21  
2      22  
3      15  
4      22  


7,5,3
2,6,1
8,4,


Question 2. Écrivez des fonctions qui prennent un taquin comme entrée (un état) et calculent h3
et h4. Attention, la tuile vide ne doit pas être considérée dans les calcules.

In [19]:
def h3(state, goal_state):
  distance = 0
  for i in range(len(state)):
      for j in range(len(state[0])):
          if state[i][j] != 0:
              goal_row, goal_col = next(
                  (r, c)
                  for r, row in enumerate(goal_state)
                  for c, val in enumerate(row)
                  if val == state[i][j]
              )
              distance += abs(i - goal_row) + abs(j - goal_col)
  for i in range(2):
      if(i*3<state[i][0]<(i+1)*3):
        distance += 2 if(state[i][0]>state[i][1]) else 0
        distance += 2 if(state[i][0]>state[i][2]) else 0
        distance += 2 if(state[i][1]>state[i][2]) else 0
      if(i*3<state[0][i]<(i+1)*3):
        distance += 2 if(state[0][i]>state[1][i]) else 0
        distance += 2 if(state[0][i]>state[2][i]) else 0
        distance += 2 if(state[1][i]>state[2][i]) else 0

  return distance

print(h3(intial_state_test, goal_state))

16


In [5]:
def h4(state, goal_state):
  return np.sum(
      [
          0.8 if state[i][j]==5 else 1 if state[i][j] in [1,3,7] else 1.2
          for i in range(len(state))
          for j in range(len(state[0]))
          if state[i][j] != goal_state[i][j] and state[i][j] != 0
      ]
  )

print(h4(intial_state_test, goal_state))

7.4


Question 3. Importer et analyser le dataset fourni. Créer trois datasets contenant soit les états
initiaux, soit les heuristiques, soit les deux.

In [59]:
from sklearn.model_selection import train_test_split

dataset1 = taquins[["col0","col1","col2","col3","col4","col5","col6","col7","col8","target"]]
dataset2 = taquins[["h1","h2","h3","h4","target"]]
dataset3 = taquins

datasets = [dataset1, dataset2, dataset3]

train_test_sets = [
    train_test_split(i.iloc[:,:-1], i.iloc[:,-1], test_size=0.2, random_state=0)
    for i in datasets
] # X_train, X_test, y_train, y_test

print(train_test_sets)

[[     col0  col1  col2  col3  col4  col5  col6  col7  col8
687     0     3     8     4     6     7     5     1     2
500     1     3     6     7     2     8     5     0     4
332     8     2     3     1     5     4     0     6     7
979     2     4     8     1     7     5     3     6     0
817     1     5     6     7     8     4     0     2     3
..    ...   ...   ...   ...   ...   ...   ...   ...   ...
835     7     5     6     1     4     2     3     8     0
192     6     0     8     3     5     7     1     4     2
629     8     3     0     4     1     6     7     5     2
559     5     2     4     3     6     8     7     1     0
684     3     6     8     2     0     4     5     1     7

[800 rows x 9 columns],      col0  col1  col2  col3  col4  col5  col6  col7  col8
993     5     2     0     6     8     3     1     7     4
859     1     5     6     3     7     0     4     2     8
298     3     5     6     1     7     8     2     4     0
553     8     4     3     1     0     7     6

Question 4. Optimiser les principaux algorithmes vus en cours afin qu’ils soient le plus adaptés
possible aux datasets proposés.

In [104]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV

df_result = pd.DataFrame(
    columns=["Algo", "Dataset ID", "MSE_train", "MSE_test"]
).astype(
    {
        "Algo": str,
        "Dataset ID": int,
        "MSE_train": float,
        "MSE_test": float,
    }
)

def optimize_model(model, train_test_set, param_distributions, id, df_result):
    X_train, X_test, y_train, y_test = train_test_set
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    mse_train = mean_squared_error(y_train, y_train_pred)
    mse_test = mean_squared_error(y_test, y_test_pred)

    print(f"\n{model}\nOriginal train MSE: {mse_train}\nOriginal test MSE:  {mse_test}")

    randomized_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        scoring="neg_mean_squared_error",
        n_jobs=-1,
        cv=5,
        random_state=0,
    )
    randomized_search.fit(X_train, y_train)

    print(f"\nBest parameters: {randomized_search.best_params_}")
    best_model = randomized_search.best_estimator_

    print(
        f"Optimized train MSE: {best_model.score(X_train, y_train)}\nOptimized test MSE:  {best_model.score(X_test, y_test)}\n"
    )
    print("-"*100)

    df_result = pd.concat(
            [
                df_result,
                pd.DataFrame(
                    [[
                        model,
                        id,
                        best_model.score(X_train, y_train),
                        best_model.score(X_test, y_test)
                    ]],
                    columns=df_result.columns,
                ),
            ],
            ignore_index=True,
        )
    return df_result

param_distributions_knn = {
    "n_neighbors": [int(x) for x in np.linspace(start=0, stop=500)],
    "weights": ["uniform", "distance"]
}
param_distributions_decisonTree = {
    "criterion": ["squared_error", "friedman_mse", "absolute_error", "poisson"],
    "max_depth": [None] + [int(x) for x in np.linspace(start=0, stop=100)],
    "min_samples_split": [int(x) for x in np.linspace(start=0, stop=50)],
    "min_samples_leaf": [int(x) for x in np.linspace(start=0, stop=50)],
}
param_distributions_randomForest = {
    "n_estimators": [int(x) for x in np.linspace(start=0, stop=500)],
    "criterion": ["squared_error", "friedman_mse", "absolute_error", "poisson"],
    "max_depth": [None] + [int(x) for x in np.linspace(start=0, stop=100)],
    "min_samples_split": [int(x) for x in np.linspace(start=0, stop=50)],
    "min_samples_leaf": [int(x) for x in np.linspace(start=0, stop=50)],
    "max_features": [None, "sqrt", "log2"],
    "bootstrap": [True, False],
}

for i in train_test_sets:
  df_result = optimize_model(KNeighborsRegressor(), i, param_distributions_knn, 1, df_result)
  df_result = optimize_model(DecisionTreeRegressor(), i, param_distributions_decisonTree, 2, df_result)
  df_result = optimize_model(RandomForestRegressor(), i, param_distributions_randomForest, 3, df_result)



KNeighborsRegressor()
Original train MSE: 4.9029
Original test MSE:  8.542

Best parameters: {'weights': 'uniform', 'n_neighbors': 81}
Optimized train MSE: 0.07635338570279471
Optimized test MSE:  0.06603231151955102

----------------------------------------------------------------------------------------------------

DecisionTreeRegressor()
Original train MSE: 0.0
Original test MSE:  11.93

Best parameters: {'min_samples_split': 7, 'min_samples_leaf': 32, 'max_depth': 42, 'criterion': 'poisson'}
Optimized train MSE: 0.21197333798357876
Optimized test MSE:  -0.04573071485091895

----------------------------------------------------------------------------------------------------

RandomForestRegressor()
Original train MSE: 0.7935003749999998
Original test MSE:  6.236538


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
5 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/_pa


Best parameters: {'n_estimators': 448, 'min_samples_split': 39, 'min_samples_leaf': 15, 'max_features': None, 'max_depth': 77, 'criterion': 'poisson', 'bootstrap': True}
Optimized train MSE: 0.3032542834778533
Optimized test MSE:  0.13511935542475095

----------------------------------------------------------------------------------------------------

KNeighborsRegressor()
Original train MSE: 3.7413499999999997
Original test MSE:  5.085400000000001

Best parameters: {'weights': 'uniform', 'n_neighbors': 10}
Optimized train MSE: 0.42254237353811563
Optimized test MSE:  0.3709259944007185

----------------------------------------------------------------------------------------------------

DecisionTreeRegressor()
Original train MSE: 3.0561030709997623
Original test MSE:  5.394095636243159

Best parameters: {'min_samples_split': 2, 'min_samples_leaf': 9, 'max_depth': 93, 'criterion': 'squared_error'}
Optimized train MSE: 0.4517728450039793
Optimized test MSE:  0.3258061570239793

-------

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
5 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/_pa


Best parameters: {'n_estimators': 448, 'min_samples_split': 39, 'min_samples_leaf': 15, 'max_features': None, 'max_depth': 77, 'criterion': 'poisson', 'bootstrap': True}
Optimized train MSE: 0.4082302678935388
Optimized test MSE:  0.3613477416282791

----------------------------------------------------------------------------------------------------

KNeighborsRegressor()
Original train MSE: 3.6783000000000006
Original test MSE:  5.666799999999999

Best parameters: {'weights': 'uniform', 'n_neighbors': 10}
Optimized train MSE: 0.3884361229212686
Optimized test MSE:  0.21584966457133803

----------------------------------------------------------------------------------------------------

DecisionTreeRegressor()
Original train MSE: 0.0
Original test MSE:  10.52

Best parameters: {'min_samples_split': 34, 'min_samples_leaf': 29, 'max_depth': 61, 'criterion': 'poisson'}
Optimized train MSE: 0.3952666038793601
Optimized test MSE:  0.3325319294686613

---------------------------------------

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
5 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/_pa


Best parameters: {'n_estimators': 448, 'min_samples_split': 39, 'min_samples_leaf': 15, 'max_features': None, 'max_depth': 77, 'criterion': 'poisson', 'bootstrap': True}
Optimized train MSE: 0.4746131719065292
Optimized test MSE:  0.38238708309636005

----------------------------------------------------------------------------------------------------


Question 5. Présenter les résultats des différents algorithmes et choisir en justifiant le meilleur
algorithme, les meilleurs hyper-paramètres et les meilleures entrées pour prédire le nombre de
coût avant la résolution.

In [111]:
print(df_result)


# Meilleur algorithme: DecisionTreeRegressor()
# Meilleurs hyper-paramètres: {'min_samples_split': 7, 'min_samples_leaf': 32, 'max_depth': 42, 'criterion': 'poisson'}
# Meilleures entrées: les heuristiques

                                                Algo  Dataset ID  MSE_train  \
0                              KNeighborsRegressor()           1   0.076353   
1                            DecisionTreeRegressor()           2   0.211973   
2  (DecisionTreeRegressor(max_features=1.0, rando...           3   0.303254   
3                              KNeighborsRegressor()           1   0.422542   
4                            DecisionTreeRegressor()           2   0.451773   
5  (DecisionTreeRegressor(max_features=1.0, rando...           3   0.408230   
6                              KNeighborsRegressor()           1   0.388436   
7                            DecisionTreeRegressor()           2   0.395267   
8  (DecisionTreeRegressor(max_features=1.0, rando...           3   0.474613   

   MSE_test  
0  0.066032  
1 -0.045731  
2  0.135119  
3  0.370926  
4  0.325806  
5  0.361348  
6  0.215850  
7  0.332532  
8  0.382387  


Question 6. Générez entre 10 et 100 taquins (en fonction du temps que vous avez à disposition)
de manière aléatoire. Assurez-vous que ces taquins soient tous solubles

In [6]:
initial_states = [
    [[1, 2, 3], [4, 5, 0], [6, 7, 8]],
    [[1, 2, 3], [0, 5, 6], [4, 7, 8]],
    [[1, 0, 3], [4, 2, 5], [7, 8, 6]],
    [[1, 0, 3], [4, 5, 2], [7, 6, 8]],
    [[1, 0, 3], [4, 2, 5], [6, 7, 8]],
    [[1, 2, 3], [4, 0, 6], [7, 5, 8]],
    [[1, 2, 3], [0, 4, 6], [7, 5, 8]],
    [[1, 2, 3], [4, 6, 0], [7, 5, 8]],
    [[1, 3, 6], [4, 2, 5], [7, 0, 8]],
    [[1, 3, 6], [4, 2, 0], [7, 5, 8]],
    [[1, 3, 6], [4, 0, 2], [7, 5, 8]],
    [[3, 1, 2], [4, 6, 5], [7, 0, 8]],
    [[8, 1, 2], [0, 4, 3], [7, 6, 5]],
    [[1, 4, 2], [7, 0, 6], [5, 3, 8]],
    [[2, 8, 3], [1, 6, 4], [7, 0, 5]],
]

Question 7. Résolvez les taquins avec BFS, DFS, et A∗en utilisant au moins cinq heuristiques
parmi :
— h1 (nombre de tuiles mal placées)
— h2 (distance de Manhattan)
— h3 (distance de Manhattan avec conflits linéaires)
— h4 (nombre de tuiles mal placées pondérée)
— Les trois meilleures heuristiques obtenues avec les méthodes de machine learning.
Indiquez clairement les heuristiques sélectionnées et justifiez votre choix.

In [7]:
def h1(state, goal_state):
    return np.sum(
        [
            1
            for i in range(len(state))
            for j in range(len(state[0]))
            if state[i][j] != goal_state[i][j] and state[i][j] != 0
        ]
    )

def h2(state, goal_state):
    distance = 0
    for i in range(len(state)):
        for j in range(len(state[0])):
            if state[i][j] != 0:
                goal_row, goal_col = next(
                    (r, c)
                    for r, row in enumerate(goal_state)
                    for c, val in enumerate(row)
                    if val == state[i][j]
                )
                distance += abs(i - goal_row) + abs(j - goal_col)
    return distance

In [9]:
def BFS(problem, buget=float("inf")):
    root = Node(problem.initial_state)
    if problem.is_goal(root):
        return root
    frontier = [root]
    explored = []
    iteration = 0
    while frontier != [] and iteration < buget:
        iteration += 1
        node = frontier.pop(0)
        explored.append(node.state)
        for action in problem.actions(node.state):
          child_state = problem.result(node.state, action)
          if child_state not in explored:
              child = Node(child_state, node, action)
              if problem.is_goal(child.state):
                  return child.solution(), iteration
              if child not in frontier:
                  frontier.append(child)
    return None, iteration

def DFS(problem, buget=float("inf")):
    root = Node(problem.initial_state)
    if problem.is_goal(root):
        return root
    frontier = [root]
    explored = []
    iteration = 0
    while frontier != [] and iteration < buget:
        iteration += 1
        node = frontier.pop(-1)
        explored.append(node.state)
        for action in problem.actions(node.state):
            child_state = problem.result(node.state, action)
            if child_state not in explored:
                child = Node(child_state, node, action)
                if problem.is_goal(child.state):
                    return child.solution(), iteration
                if child not in frontier:
                    frontier.append(child)
    return None, iteration

def a_etoile(problem, heuristic, Max_it=float("inf")):
    # Initialisation de la file d'attente prioritaire (frontier) et de l'ensemble des nœuds explorés
    frontier = [
        (0, Node(problem.initial_state))
    ]  # Utilisation d'une liste pour la frontier
    explored = []
    it = 0

    # Boucle principale de l'algorithme A*
    while frontier and it < Max_it:
        it = it + 1
        # Trouver le nœud ayant la valeur f la plus faible dans la frontier
        min_f = float("inf")
        min_index = -1
        for i, (f, node) in enumerate(frontier):
            if f < min_f:
                min_f = f
                min_index = i

        # Retirer le nœud ayant la valeur f la plus faible de la frontier
        _, node = frontier.pop(
            min_index
        )  # We do not store the value f as we will not use it

        if problem.is_goal(node.state):
            # Solution trouvée !
            return node, it

        explored.append(node.state)

        for child in node.expand(problem):
            if child.state not in explored:
                # Calcul de la valeur f (f = g + h)
                g = child.path_cost
                h = heuristic(
                    child.state, problem.goal_state
                )  # Utilisation de la fonction heuristique spécifiée
                f = g + h

                # Ajout du nœud enfant à la frontier
                frontier.append((f, child))

    # Aucune solution trouvée
    return None, it

In [11]:
import time
initial_states = [
    [[1, 2, 3], [4, 5, 0], [6, 7, 8]],
    [[1, 2, 3], [0, 5, 6], [4, 7, 8]],
    [[1, 0, 3], [4, 2, 5], [7, 8, 6]],
    [[1, 0, 3], [4, 5, 2], [7, 6, 8]],
    [[1, 0, 3], [4, 2, 5], [6, 7, 8]],
    [[1, 2, 3], [4, 0, 6], [7, 5, 8]],
    [[1, 2, 3], [0, 4, 6], [7, 5, 8]],
    [[1, 2, 3], [4, 6, 0], [7, 5, 8]],
    [[1, 3, 6], [4, 2, 5], [7, 0, 8]],
    [[1, 3, 6], [4, 2, 0], [7, 5, 8]],
    [[1, 3, 6], [4, 0, 2], [7, 5, 8]],
    [[3, 1, 2], [4, 6, 5], [7, 0, 8]],
    [[8, 1, 2], [0, 4, 3], [7, 6, 5]],
    [[1, 4, 2], [7, 0, 6], [5, 3, 8]],
    [[2, 8, 3], [1, 6, 4], [7, 0, 5]],
]
def run_algorithms(algo, initial_states, goal_state, heuristic=None, max_it=float("inf")):
    result = []
    for initial_state in initial_states:
      problem = Taquin(initial_state, goal_state, 3)
      start_time = time.time()
      if heuristic :
        solution_node, num_explorations = algo(problem, heuristic, max_it)
        end_time = time.time()
        execution_time = end_time - start_time
        if solution_node:
          path_length = len(solution_node.solution())
        else:
            path_length = -1  # Indicate no solution found
      else:
        solution_node, num_explorations = algo(problem, max_it)
        end_time = time.time()
        execution_time = end_time - start_time

        if solution_node:
            path_length = len(solution_node)
        else:
            path_length = -1  # Indicate no solution found


      result.append(
            {
                "initial_state": initial_state,
                "algorithm": algo.__name__,  # Get the name of the function
                "num_explorations": num_explorations,
                "path_length": path_length,
                "execution_time": execution_time,
            }
        )
    return result

results_BFS = run_algorithms(BFS, initial_states, goal_state, max_it=5000)
results_DFS = run_algorithms(DFS, initial_states, goal_state, max_it=5000)
results_A_H1 = run_algorithms(a_etoile, initial_states, goal_state, h1, max_it=5000)
results_A_H2 = run_algorithms(a_etoile, initial_states, goal_state, h2, max_it=5000)
results_A_H3 = run_algorithms(a_etoile, initial_states, goal_state, h3, max_it=5000)
results_A_H4 = run_algorithms(a_etoile, initial_states, goal_state, h4, max_it=5000)

Question 8. Créez trois tableaux sous format DataFrame où :
1. Le premier tableau contient le numéro de l’état initial et le nombre de nœuds explorés par
chaque algorithme.
2. Le deuxième tableau contient le numéro de l’état initial et le temps d’exécution de chaque
algorithme.
3. Le troisième tableau contient le numéro de l’état initial et la longueur du chemin trouvé par
chaque algorithme pour atteindre l’état but.

In [13]:
Data = []
for i in range(len(initial_states)):
    Data.append(
        [
            int(i + 1),
            results_BFS[i]["num_explorations"],
            results_DFS[i]["num_explorations"],
            results_A_H1[i]["num_explorations"],
            results_A_H2[i]["num_explorations"],
            results_A_H3[i]["num_explorations"],
            results_A_H4[i]["num_explorations"],
            results_BFS[i]["path_length"],
            results_DFS[i]["path_length"],
            results_A_H1[i]["path_length"],
            results_A_H2[i]["path_length"],
            results_A_H3[i]["path_length"],
            results_A_H4[i]["path_length"],
            round(results_BFS[i]["execution_time"], 2),
            round(results_DFS[i]["execution_time"], 2),
            round(results_A_H1[i]["execution_time"], 2),
            round(results_A_H2[i]["execution_time"], 2),
            round(results_A_H3[i]["execution_time"], 2),
            round(results_A_H4[i]["execution_time"], 2),
        ]
    )

df = pd.DataFrame(
    Data,
    columns=[
        "State",
        "Nodes explored BFS",
        "Nodes explored DFS",
        "Nodes explored A*H1",
        "Nodes explored A*H2",
        "Nodes explored A*H3",
        "Nodes explored A*H4",
        "Path length BFS",
        "Path length DFS",
        "Path length A*H1",
        "Path length A*H2",
        "Path length A*H3",
        "Path length A*H4",
        "Execution Time BFS",
        "Execution Time DFS",
        "Execution Time A*H1",
        "Execution Time A*H2",
        "Execution Time A*H3",
        "Execution Time A*H4",
    ],
)

Question 9. Imprimez les trois tableaux en utilisant tabulate avec l’option tablefmt=‘pretty’.

In [14]:
from tabulate import (
    tabulate,
)  ## La librairie Tabulate doit être installée. Si vous ne l'avez pas, vous pouvez simplement utiliser "print(df)" pour visualiser le tableau.

print(tabulate(df, headers="keys", tablefmt="pretty"))

+----+-------+--------------------+--------------------+---------------------+---------------------+---------------------+-----------------+-----------------+------------------+------------------+------------------+--------------------+--------------------+---------------------+---------------------+---------------------+
|    | State | Nodes explored BFS | Nodes explored DFS | Nodes explored A*H1 | Nodes explored A*H2 | Nodes explored A*H4 | Path length BFS | Path length DFS | Path length A*H1 | Path length A*H2 | Path length A*H4 | Execution Time BFS | Execution Time DFS | Execution Time A*H1 | Execution Time A*H2 | Execution Time A*H4 |
+----+-------+--------------------+--------------------+---------------------+---------------------+---------------------+-----------------+-----------------+------------------+------------------+------------------+--------------------+--------------------+---------------------+---------------------+---------------------+
| 0  |  1.0  |       1665.0 

Question 10. En proposant différentes statistiques sur ce tableaux, analyser la performance de
ces heuristiques.